In [ ]:
!pip install -q -U qdrant-client stamina

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 20.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.1 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.29.1 which is incompatible.


In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from io import BytesIO
from qdrant_client import QdrantClient
from qdrant_client.http import models
from IPython.display import display
from io import BytesIO
import base64
from PIL import Image
import torch
import cv2
import numpy as np
from pathlib import Path
from typing import List, Dict
from tqdm import tqdm
import json
import os

def encode_base64_image(image):
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    return str(base64.b64encode(buffered.getvalue()), "utf-8")


def decode_base64_image(base64_string):
    # Decode the base64 string into bytes
    image_data = base64.b64decode(base64_string)

    # Convert the bytes back into an image
    buffered = BytesIO(image_data)
    image = Image.open(buffered)

    return image


def resize_image(image, max_height=800):
    width, height = image.size
    if height > max_height:
        ratio = max_height / height
        new_width = int(width * ratio)
        new_height = int(height * ratio)
        return image.resize((new_width, new_height))
    return image


def save_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)

def load_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)

def combine_json_files(directory: str):
    combined_data = []

    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            data = load_json(file_path)
            combined_data.extend(data)

    return combined_data
def convert_sparse_output(sparse_output):
    sparse_transformed = []
    for dic in sparse_output:
        result = {"values": [], "indices": []}
        for idx, val in dic.items():
            result["values"].append(val)
            result["indices"].append(int(idx))
        sparse_transformed.append(result)
    return sparse_transformed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
collection_name = "embedded-manuals"
QDRANT_API_KEY = ""
qdrant_client = QdrantClient(
    url="https://568e9069-f3eb-4ee2-8e22-52e7bad7d3b7.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key=QDRANT_API_KEY,
)

qdrant_client.recreate_collection(
    collection_name=collection_name,
    on_disk_payload=True,
    vectors_config = {
        "dense": models.VectorParams(
        size=1024,
        distance=models.Distance.DOT,
    ),  "multi": models.VectorParams(
        size=128,
        distance=models.Distance.DOT,
        multivector_config=models.MultiVectorConfig(
            comparator=models.MultiVectorComparator.MAX_SIM
        ),
    )
      },
    sparse_vectors_config = {
        "sparse": models.SparseVectorParams(
            index=models.SparseIndexParams(
                on_disk=False,
            )
        ),
        "sparse_headings": models.SparseVectorParams(
            index=models.SparseIndexParams(
                on_disk=False,
            ))
    }
)

<ipython-input-15-10c11b4191bd>:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [ ]:
all_data = load_json("/content/drive/MyDrive/Embeddings/Neue/embedded-manuals.json")

In [ ]:
qdrant_documents_multi = []
iddd = 0
for page in all_data:
    iddd += 1
    qdrant_documents_multi.append(
          models.PointStruct(
              id= iddd,
              vector={
                "dense": page["dense_embeddings"],
                "sparse": page["sparse_embeddings"],
                "multi": page["colbert_embeddings"],
                "sparse_headings": page["sparse_headings"],

              },
              payload={
                  "title": page["title"],
                  "page_number": page["page_number"],
                  "text": page["text"],
                  "headings": page["headings"],
                  "language": page["language"],
                  "label": page["label"],
                  "image": page["base64"],

              },
          )
      )


In [ ]:
import stamina


@stamina.retry(on=Exception, attempts=3)
def upsert_to_qdrant(batch):
    try:
        qdrant_client.upsert(
            collection_name=collection_name,
            points=batch,
            wait=False,
        )
    except Exception as e:
        print(f"Error during upsert: {e}")
        return False
    return True

In [ ]:
batch_size = 10

with tqdm(total=len(qdrant_documents_multi), desc="Upserting Progress") as pbar:
    for i in range(0, len(qdrant_documents_multi), batch_size):
        points = qdrant_documents_multi[i : i + batch_size]

        try:
            upsert_to_qdrant(points)
        except Exception as e:
            print(f"Error during upsert: {e}")
            continue

        pbar.update(batch_size)

print("Upserting complete!")

Upserting Progress: 1080it [00:31, 33.88it/s]

Upserting complete!


In [ ]:
ids_json = []
i = 0
for page in all_data:
    i += 1
    ids_json.append(
        {
            "id": i,
            "title": page["title"],
            "page_number": page["page_number"],
        }
    )
save_json(ids_json, "/content/drive/MyDrive/Embeddings/Neue/ids-pages.json")
print(ids_json[:5])

[{'id': 1, 'title': 'MiR 24V Battery Troubleshooting and Technical Guide 2.1_en', 'page_number': 1}, {'id': 2, 'title': 'MiR 24V Battery Troubleshooting and Technical Guide 2.1_en', 'page_number': 2}, {'id': 3, 'title': 'MiR 24V Battery Troubleshooting and Technical Guide 2.1_en', 'page_number': 3}, {'id': 4, 'title': 'MiR 24V Battery Troubleshooting and Technical Guide 2.1_en', 'page_number': 4}, {'id': 5, 'title': 'MiR 24V Battery Troubleshooting and Technical Guide 2.1_en', 'page_number': 5}]


# Chunks

In [ ]:
collection_name = "chunked-and-embedded-manuals"
QDRANT_API_KEY = ""
qdrant_client = QdrantClient(
     url="https://c1c6b76d-3e82-4265-8850-542922eeb4b9.us-east4-0.gcp.cloud.qdrant.io:6333/dashboard#/collections",
     api_key=QDRANT_API_KEY,
 )

qdrant_client.recreate_collection(
    collection_name=collection_name,
    on_disk_payload=True,
    vectors_config = {
        "dense": models.VectorParams(
        size=1024,
        distance=models.Distance.DOT,
    ),  "multi": models.VectorParams(
        size=128,
        distance=models.Distance.DOT,
        multivector_config=models.MultiVectorConfig(
            comparator=models.MultiVectorComparator.MAX_SIM
        ),
    )
      },
    sparse_vectors_config = {
        "sparse": models.SparseVectorParams(
            index=models.SparseIndexParams(
                on_disk=False,
            )
        ),
        "sparse_headings": models.SparseVectorParams(
            index=models.SparseIndexParams(
                on_disk=False,
            ))
    }
)

<ipython-input-11-6f5c86a6d9d2>:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [ ]:
all_data_chunks = load_json("/content/drive/MyDrive/Embeddings/Neue/chunked-and-embedded-manuals.json")

In [ ]:
qdrant_documents_multi = []
i = 0
for chunk in all_data_chunks:
    i += 1
    qdrant_documents_multi.append(
          models.PointStruct(
              id= i,
              vector={
                "dense": chunk["dense_embeddings"],
                "sparse": chunk["sparse_embeddings"],
                "multi": chunk["colbert_embeddings"],
                "sparse_headings": chunk["sparse_headings"]

              },
              payload={
                  "title": chunk["title"],
                  "page_number": chunk["page_number"],
                  "text": chunk["text"],
                  "headings": chunk["headings"],
                  "language": chunk["language"],
                  "label": chunk["label"],
                  "image": chunk["base64"],

              },
          )
      )


In [ ]:
import stamina


@stamina.retry(on=Exception, attempts=3)
def upsert_to_qdrant(batch):
    try:
        qdrant_client.upsert(
            collection_name=collection_name,
            points=batch,
            wait=False,
        )
    except Exception as e:
        print(f"Error during upsert: {e}")
        return False
    return True

In [ ]:
batch_size = 10

with tqdm(total=len(qdrant_documents_multi), desc="Upserting Progress") as pbar:
    for i in range(0, len(qdrant_documents_multi), batch_size):
        points = qdrant_documents_multi[i : i + batch_size]

        try:
            upsert_to_qdrant(points)
        except Exception as e:
            print(f"Error during upsert: {e}")
            continue

        pbar.update(batch_size)

print("Upserting complete!")

Upserting Progress: 3120it [01:44, 29.88it/s]

Upserting complete!
